# Instruction Fine-tuning

This notebook demonstrates fine-tuning a model on instruction data (Alpaca-style).


In [13]:
import sys
sys.path.append('..')

import torch
import tiktoken
from src.model import GPTModel, GPT_CONFIG_124M
from src.finetune import train_model_simple, generate_text_simple, text_to_token_ids, token_ids_to_text
from src.formatter import format_input_advanced
from torch.utils.data import DataLoader


In [14]:
import json
import os
import urllib.request
import ssl
import certifi

def download_and_load_file(file_path, url):
    if not os.path.exists(file_path):
        # Create verified SSL context
        ssl_context = ssl.create_default_context(cafile=certifi.where())
        with urllib.request.urlopen(url, context=ssl_context) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    
    with open(file_path, "r") as file:
        data = json.load(file)
    return data

file_path = "../data/alpaca-instruction-data.json"
url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))


Number of entries: 52002


In [15]:
print("Example entry:\n", data[0])

Example entry:
 {'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


In [16]:
# Configure the prompt style - choose from: 'enhanced', 'chatml', 'task_aware', 'cot', 'structured'
PROMPT_STYLE = 'enhanced'

def format_input(entry):
    """
    Format instruction using the advanced formatter module.
    
    Available styles:
    - 'enhanced': Improved Alpaca format (recommended)
    - 'chatml': ChatML-style format (ChatGPT/GPT-4 style)
    - 'task_aware': Adapts based on task type
    - 'cot': Chain-of-thought for reasoning tasks
    - 'structured': Encourages structured outputs
    """
    return format_input_advanced(entry, style=PROMPT_STYLE)

In [17]:
# Demo: Compare different formatting styles
print("Testing different prompt formats with the first data entry:\n")
print("="*80)

example = data[0]

styles = ['enhanced', 'chatml', 'task_aware', 'cot', 'structured']

for style in styles:
    print(f"\n📝 Style: {style.upper()}")
    print("-"*80)
    formatted = format_input_advanced(example, style=style)
    print(formatted)
    print("="*80)


Testing different prompt formats with the first data entry:


📝 Style: ENHANCED
--------------------------------------------------------------------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.
### Response:


📝 Style: CHATML
--------------------------------------------------------------------------------
<|im_start|>system
You are a helpful AI assistant that follows instructions precisely.<|im_end|>
<|im_start|>user
Give three tips for staying healthy.<|im_end|>
<|im_start|>assistant


📝 Style: TASK_AWARE
--------------------------------------------------------------------------------
[SYSTEM] You are a helpful assistant. Follow instructions carefully and provide accurate responses.

[TASK] Give three tips for staying healthy.

[RESPONSE]

📝 Style: COT
--------------------------------------------------------------------------------
Below is an instruction tha